In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import typing
from typing import List, Set, Dict, TypeVar, Union, Optional, Callable

from copy import deepcopy

from funcy import walk

from bidict import bidict
import bidict as bd


A = TypeVar('A')

In [3]:
import z3

# Array representations

## "Array" does *not* mean what you think it does

"Arrays" in an SMT context ("the theory of arrays with extensional equality") are essentially the same (they reduce to) **uninterpreted functions with finite domain of definition** and a couple of very simple axioms that SMT solvers have some special support for (as well as usually some limited support for extensions of that theory):
 - 'Select'ing the value at index `x` is like applying a function to `x`: `A[x] == f(x)`.
 - "Arrays" have a domain ("index") type and a codomain ("value") type.
 - "Arrays" are *total* over their domain: they have as many values as they do domain elements.
 
"Arrays" in an SMT context are **not** like your typical array from a general programming context in that:
   - We cannot in general query the length of the array per se.
     - Getting a concrete length requires extracting that information from somewhere or something related to the domain sort (if that's possible).
   - There is no way to "update" a declared array: you instead declare that a new array is the result of updating some other array.

(See [this introduction to the theory of arrays](http://staff.ustc.edu.cn/~bjhua/courses/theory/2021/ref/chap-array.pdf) for more information, including the three axioms that characterize arrays in particular.)

Basic code for manipulating arrays:

Declare a symbolic array:

In [4]:
# Declare an array "A", with the domain of the entire set of integers and a 
# codomain of the booleans.

A = z3.Array('A', z3.IntSort(), z3.BoolSort()) 

# - "Sorts" are analogous to types. 
# - The foundation of SMT solvers is many-sorted FOL and quantifier free 
#   fragments thereof. 
# - z3 has a limited number of built in sorts with special support (but you can 
#   define your own).

Select a value at a (concrete or symbolic) index:

In [5]:
# Select the value at a concrete index
A[2]

# Select the value at the x'th index
x = z3.Int('x') 
A[x]
z3.Select(A, x)

A[2]

A[x]

A[x]

Create a new array that is the result of updating an existing one:

In [6]:
# Create an array that is the result of making a copy of A, but storing 
# "True" at index 2
z3.Store(A, 2, z3.BoolVal(True))

Store(A, 2, True)

In [7]:
# Retrieve the value at index x from an array defined to have `True` at `x`
#  (NB No solver or model is involved here, and the result type here (despite
#  appearances) is not a concrete python boolean: "simplify" is a function that 
#  heuristically transforms abstract syntax trees, but it's an idiomatic/accept-
#  able 'evaluator' for stuff like this.
z3.simplify(z3.Select(z3.Store(A, x, z3.BoolVal(True)), x))

True

Create a new array that has a default value everywhere:

In [8]:
z3.K(z3.IntSort(), False)

K(Int, False)

Getting concrete arrays out, given constraints:

In [9]:
# each of these is (separately) 
#  - creating a solver instance
#  - adding the couple of constraints indicated
#  - checking if there is a model under the constraints, and if so returning it
z3.solve(A[0] == False, A[2] == False, A[344] == False) # -> start with an all-false array.
z3.solve(A[0] == False, A[2] == True, A[344] == False)  # -> start with an all-false array and then just modify 2
z3.solve(A[0] == False, A[2] == True, A[344] == True)   # -> start with an all-true array and then just modify 0

[A = K(Int, False)]
[A = Store(K(Int, False), 2, True)]
[A = Store(K(Int, True), 0, False)]


## Encoding (sub)sets using arrays

There are a few ways of encoding finite sets over some (possibly infinite) universe $U$ of possible values entirely inside of `z3`. 
 1. The template above - an array with "index" sort $U$ and "values" sort $\text{BoolSort}$ -  is the idiomatic one with (limited) special support. **We explore that immediately below.**
    - You can declare and work with symbolic subsets whose maximum size does not have to be a concrete value known ahead of time.
    - The complexity of the theory needed to capture your index and values sorts (separately and in combination) affects the decidability (and performance) of constraints involving arrays.
 2. You can also create a concrete (python) list of symbolic (z3) Booleans.
    - You must now encode all desired set operations (as z3 relations) yourself.
      - This may be simpler in some ways and more complicated in others.
      - If you are not used to writing in a functional style (or carefully doing the opposite), this may be easy to do incorrectly in such a way that you either don't understand why z3's results don't make sense, or mysterious error messages.
      - Regardless, all the normal confusions of managing references vs. values (and referential vs extensional equality) are obscured by being behind a C++ implementation, and even setting that aside, parallel issues show up in new ways in the context of solver variables, and still require some effort to navigate carefully.
    - With regard to performance, my possibly incorrect understanding is that, given a universe of $n$ possible values (or subsets of size at most $n$), you will often have set operation implementations that add $O(n)$ or $O(n^2)$ constraints where `z3`'s array theory-backed version would add e.g. $O(1)$ constraints with a theory-of-arrays-specific implementation.
 3. You can a create [Church-encoding](https://en.wikipedia.org/wiki/Church_encoding)-like [representation of a subset in terms of a composition of functions that each essentially represent a membership query](https://stackoverflow.com/a/70250257).
    - Python may be impractically fussy about nested functions (or operations on them) if the nesting gets too deep, and that may happen surprisingly soon.
    - You still have to implement set operations yourself.
    - (In principle, you may also be able to use this representation 'inside' of z3, but you'd need a good reason to do this.)

 
 

(`cvc4`/`cvc5` have dedicated theories for finite sets and relations that are more expressive and ergonomic.)

### Arrays as subsets - no magic

In [10]:
# Declare a custom sort consisting of exactly three values


concrete_color_values = ('red', 'green', 'blue')

# References to the sort and to the three possible ways of constructing a value
# of sort 'Color'
ColorSort, (red, green, blue) = z3.EnumSort(name='Color', values=concrete_color_values)

# This lets us declare a new symbolic constant of sort 'Color' - e.g. if we want
# to create concrete (python) lists of symbolic colors.
def Color(identifier):
  return Const(identifier, ColorSort)


z3_color_values = (red, green, blue)
color_map = bidict({'red': red, 'green': green, 'blue': blue})

In [11]:
# A symbolic subset is an array that maps each element of the index sort to a 
# a boolean value, indicating whether that element is included.
some_colors = z3.Array('some_colors', ColorSort, z3.BoolSort())

# Note that right now, this array is totally unconstrained: any possible 
# assignment of truth value to each of the three color-indexed symbolic booleans
# is a valid assignment, right now.

In [12]:
z3.solve(some_colors[red] == True, some_colors[blue] == False)

[some_colors = Store(K(Color, False), red, True)]


Let's create a symbolic subset that definitely contains red, but isn't otherwise constrained.

In [13]:
# This is the symbolic subset that is defined to be the result of
#  - taking the array `some_colors`...
#  - and storing `z3.BoolVal(True)`...
#  - at the index `red`
r = z3.Store(some_colors, red, z3.BoolVal(True))

# `select` symbolically retrieves the value stored at `red` in `r`.
z3.simplify(z3.Select(r, red))

True

In [14]:
# silly trick for making the model show us all assignments, including the ones
# we specified.
X = z3.Array('X', ColorSort, z3.BoolSort())
z3.solve(X == r)

[some_colors = K(Color, False),
 X = Store(K(Color, False), red, True)]


Array definitions don't (at this level) "simplify" their definitional history:
 - Look below at `X` in the model below: `green` is added to the subset twice.

In [15]:
rg = z3.Store(r, green, z3.BoolVal(True))
r_noB = z3.Store(r, blue, z3.BoolVal(False))

z3.simplify(z3.Select(r_noB, blue))

z3.solve(X == rg, X == r_noB)

False

[some_colors = Store(K(Color, False), green, True),
 X = Store(Store(Store(K(Color, False), green, True),
                 red,
                 True),
           green,
           True)]


### z3's subset interface

z3 has a number of functions for working with array-based representations of finite sets.

In [16]:
empty_colors = z3.EmptySet(ColorSort)
just_red     = z3.SetAdd(empty_colors, red)
just_green   = z3.Store(empty_colors, green, z3.BoolVal(True)) # equivalent
just_blue    = z3.SetAdd(empty_colors, blue)

# 'No solution' - because it's empty *by definition*
z3.solve(z3.IsMember(red, empty_colors)) 

# '[]' - because there are no *free variables* that need to be assigned: every 
# possible assignment of values to the variables for green and blue satisfies
# the only constraint we've asked the solver to solve for. Why? Because `red` 
# is a member of `just_red` *by definition* - `just_red` is *defined* to be the
# result of adding red to the empty set.
z3.solve(z3.IsMember(red, just_red))

no solution
[]


In [17]:
# yields model where all colors in some_colors are true
z3.solve(z3.IsMember(red, some_colors)) 

# yields model where X has all colors are true, except blue
z3.solve(z3.IsMember(red, some_colors), X == z3.SetIntersect(some_colors, r_noB)) 

[some_colors = K(Color, True)]
[some_colors = K(Color, True),
 X = Store(K(Color, True), blue, False)]


In [18]:
# solve_([z3.IsMember(red, some_colors), X == z3.SetIntersect(some_colors, r_noB)])[some_colors]

### Concrete Boolean lists of symbolic Booleans

Below is a parallel interface where the sequence of symbolic Booleans is a Python list.

In fact, it is slightly more than one interface - per the earlier note, there are two basic ways to use this interface:
 - one attempts to mirror the `z3` subset functions, and might be more complicated to reason about or have unexpected behavior.
 - the other is more relationally-flavored and works with more distinct Boolean variables.

Note that the theory of arrays *with extensional equality* avoids some of this confusion by asserting that any two arrays that have the same domains of definition (indices) and that have the same values for each point in their point are considered indistinguishable. 
Explicitly modeling subsets with lists of references to Boolean variables is in some ways simpler (locally less mysterious), but definitely introduces sources of reference-related confusion that you must make some effort to avoid or prevent.

In [19]:
def colorIndex(color) -> int:
  if type(color) == str:
    assert color in concrete_color_values
    return concrete_color_values.index(color)
  elif type(color) is z3.DatatypeRef:
    return concrete_color_values.index(color_map.inverse[color])
  else:
    raise Exception(f"Unknown type {color}")

    
Bool = Union[bool, z3.BoolRef]

def freshSet(n: int, prefix: str) -> List[z3.BoolRef]:
  return [z3.Bool(f"{prefix}__{i}") for i in range(n)]
#   return z3.BoolVector(prefix, n) # equivalent

#returns a list of constraints on the elements of `subset`
def isEmptySet(subset: List[Bool]) -> Bool:
  return [z3.Not(b) for b in subset]

def emptySet(n: int, prefix: str) -> List[z3.BoolRef]:
  return freshEmptySet_declaration(n, prefix)

#returns a list of boolean constants all declared to be false
def freshEmptySet_declaration(n: int, prefix: str) -> List[z3.BoolRef]:
  return [z3.BoolVal(False) for i in range(n)]

# Returns a list of constraints on elments of a fresh subset that will 
# be difficult to access. (This function is more meant as an illustration of the
# difference between two seemingly equivalent ways with some downstream conseq-
# uences for how much work you do or don't have to do to interpret solver 
# output.)
def freshEmptySet_constraint(n: int, prefix: str) -> List[z3.BoolRef]:
  return [z3.Not(b) for b in freshSet(n, prefix)]


def setAdd_faithfulTranslation(universe: Callable[[A], int]):
  def setAdd_forUniverse(subset: List[Bool], value: A) -> List[Bool]:
    target_index: int = universe(value)
    
    # Naively correct looking implementation...
    # This is sketchy in a normal python way: any reference to a `subset` 
    # argument list elsewhere that's then used to alter the contents or declare
    # constraints on those symbolic booleans will also affect this list.
#     return [subset[i] if (i != target_index) else z3.BoolVal(True)
#             for i in range(len(subset))]

    # Also potentially sketchy because who knows if deepcopy is actually what
    # you want. Consider: do constraints already placed on subset transfer
    # to deep-copies of its variables? That really depends on knowing how
    # constraints are represented by z3 and what you're trying to do.
    return [deepcopy(subset[i]) if (i != target_index) else z3.BoolVal(True)
           for i in range(len(subset))]
  return setAdd_forUniverse

def isSetAddOf(universe: Callable[[A], int]):
  def isSetAddOf_forUniverse(updated: List[Bool], 
                             subset: List[Bool], 
                             value: A) -> Bool:
    assert len(updated) == len(subset)
    target_index: int = universe(value)
    
    return [(updated[i]) == subset[i] if i != target_index else updated[i] == True
            for i in range(len(updated))]
  return isSetAddOf_forUniverse

def isSetUnionOf(subset: List[Bool], left: List[Bool], right: List[Bool]):
  assert len(subset) == len(left) == len(right)
  return z3.And([subset[i] == z3.Or(left[i], right[i])
                  for i in range(len(subset))])

setAdd = setAdd_faithfulTranslation


def setDel_faithfulTranslation(universe: Callable[[A], int]):
  def setDel_forUniverse(subset: List[Bool], value: A) -> List[Bool]:
    target_index: int = universe(value)
    
    # Naively correct looking implementation...
    # This is sketchy in a normal python way: any reference to a `subset` 
    # argument list elsewhere that's then used to alter the contents or declare
    # constraints on those symbolic booleans will also affect this list.
#     return [subset[i] if (i != target_index) else z3.BoolVal(False)
#             for i in range(len(subset))]

    # Also potentially sketchy because who knows if deepcopy is actually what
    # you want. Consider: do constraints already placed on subset transfer
    # to deep-copies of its variables? That really depends on knowing how
    # constraints are represented by z3 and what you're trying to do.
    return [deepcopy(subset[i]) if (i != target_index) else z3.BoolVal(False)
            for i in range(len(subset))]
  return setDel_forUniverse

def isSetDelOf(universe: Callable[[A], int]):
  def isSetDelOf_forUniverse(updated: List[Bool], 
                             subset: List[Bool], 
                             value: A) -> Bool:
    assert len(updated) == len(subset)
    target_index: int = universe(value)
    
    return [(updated[i]) == subset[i] if i != target_index else updated[i] == False
            for i in range(len(updated))]
  return isSetDelOf_forUniverse

def isSetDiffOf(subset: List[Bool], left: List[Bool], right: List[Bool]):
  assert len(subset) == len(left) == len(right)
  return z3.And([subset[i] == z3.And(          left[i]
                                     , z3.Not(right[i]) )
                  for i in range(len(subset))])


setDel = setDel_faithfulTranslation

In [20]:
empty_colors_1 = emptySet(3, "EmptySet3")

# Expected: any assignment to any of the variables is fine.
some_colors_   = freshSet(3, "some_colors_1")
z3.solve(some_colors_) 

# Expected: any assignment to any variable except for red (index 0) is fine.
r_             = setAdd(colorIndex)(some_colors_, red)
z3.solve(r_) # fine

# Expected: any assignment to any variable except for red or green (index 1) is 
# fine.
rg_            = setAdd(colorIndex)(r_, green)
z3.solve(rg_) # fine

# Expected: 
# - X definitely has red and green and definitely does not have blue
r_noB_1        = setDel(colorIndex)(r_, blue)
X_             = freshSet(3, "X_")
z3.solve([X_[i] == rg_[i] for i in range(len(X_))] + \
         [X_[i] == r_noB_1[i] for i in range(len(X_))])

[some_colors_1__0 = True,
 some_colors_1__2 = True,
 some_colors_1__1 = True]
[some_colors_1__2 = True, some_colors_1__1 = True]
[some_colors_1__2 = True]
[some_colors_1__2 = False,
 X___2 = False,
 X___0 = True,
 X___1 = True,
 some_colors_1__1 = True]


In [21]:
def solve_(assertions):
  solver = z3.Solver()
  for each in assertions:
    solver.add(each)
  if solver.check():
    return solver.model()
  else:
    return None

assertions = []

empty_colors_2 = freshSet(3, "empty_colors_2")
empty_colors_2_isEmpty = isEmptySet(empty_colors_2)
# Expected: empty_colors_2 is empty (all values in it are false)
assertions.append(empty_colors_2_isEmpty)
print(solve_(assertions)) # fine

some_colors_2  = freshSet(3, "some_colors_2")
# Expected: same as before + some_colors_2 is totally unconstrainted
# is fine
print(solve_(assertions)) # fine

[empty_colors_2__2 = False,
 empty_colors_2__0 = False,
 empty_colors_2__1 = False]
[empty_colors_2__2 = False,
 empty_colors_2__0 = False,
 empty_colors_2__1 = False]


In [22]:
r_2            = freshSet(3, "r_2")
r_2_hasRed     = isSetAddOf(colorIndex)(r_2, some_colors_2, red)
# Expected: same as before + any assignment to r_2 is fine - except red must be
# present.
assertions = assertions + r_2_hasRed
print(solve_(assertions)) # fine

[r_2__0 = True,
 r_2__1 = False,
 some_colors_2__1 = False,
 empty_colors_2__0 = False,
 empty_colors_2__1 = False,
 r_2__2 = False,
 empty_colors_2__2 = False,
 some_colors_2__2 = False]


In [23]:
rg_2           = freshSet(3, "rg_2")
rg_2_hasRed_hasGreen = isSetAddOf(colorIndex)(rg_2, r_2, green)
assertions = assertions + rg_2_hasRed_hasGreen
print(solve_(assertions)) # fine

[rg_2__1 = True,
 r_2__0 = True,
 r_2__1 = False,
 some_colors_2__1 = False,
 empty_colors_2__0 = False,
 empty_colors_2__1 = False,
 rg_2__0 = True,
 r_2__2 = False,
 empty_colors_2__2 = False,
 rg_2__2 = False,
 some_colors_2__2 = False]


In [24]:
r_noB_2        = freshSet(3, "r_noB_2")
r_noB_2_hasRed_hasNoBlue = isSetDelOf(colorIndex)(r_noB_2, r_2, blue)
assertions = assertions + r_noB_2_hasRed_hasNoBlue
print(solve_(assertions)) # fine

[r_noB_2__1 = False,
 some_colors_2__2 = False,
 r_2__1 = False,
 some_colors_2__1 = False,
 empty_colors_2__1 = False,
 r_noB_2__0 = True,
 r_2__2 = False,
 empty_colors_2__2 = False,
 rg_2__2 = False,
 r_2__0 = True,
 empty_colors_2__0 = False,
 rg_2__0 = True,
 r_noB_2__2 = False,
 rg_2__1 = True]


# Symbolic feature vector representations

### Array based representations

#### Direct representation

The sort of a feature vector is an array 
 - with index sort taken from an enumeration of feature labels (or the integers).
 - with values sort taken from an enumeration of feature values.

In [25]:
FeatureValueSort, (plus, zero, minus) = z3.EnumSort('FVal', ('+', '0', '-'))

def FVal(identifier: str):
  return z3.Const(identifier, FeatureValueSort)

In [26]:
NUM_FEATURES = M = 3

FeatureSort3, (FA, FB, FC) = z3.EnumSort('Feature3', 
                                         ('FA', 
                                          'FB', 
                                          'FC'))
FeatureSort3_constants = (FA, FB, FC)

FeatureSort4, (high, low, front, back) = z3.EnumSort('Feature4', 
                                                     ('high', 
                                                      'low', 
                                                      'front', 
                                                      'back'))
FeatureSort4_constants = (high, low, front, back)


def Feature_(featureSort):
  def Feature__(identifier: str):
    return z3.Const(identifier, featureSort)
  return Feature__

FeatureSort = FeatureSort3 if NUM_FEATURES == 3 else FeatureSort4

Feature3 = Feature_(FeatureSort3)
Feature4 = Feature_(FeatureSort4)
Feature  = Feature_(FeatureSort)

In [27]:
def FeatureVector_(index_domain_sort = None):
  if index_domain_sort is None:
    index_domain_sort = z3.IntSort()
  def FeatureVector__(identifier: str):
    return z3.Array(identifier, index_domain_sort, FeatureValueSort)
  return FeatureVector__

def concretize(model, variable, featureValues) -> Dict[str, str]:
  modelRef = model[variable]
  domain   = modelRef.domain()
  features = [domain.constructor(i)
              for i in range(domain.num_constructors())]
  values   = [z3.simplify(modelRef[value])
              for value in featureValues]
  return dict(zip(map(str,features), map(str,values)))
  

FeatureVector3 = FeatureVector_(FeatureSort3)
FeatureVector4 = FeatureVector_(FeatureSort4)
FeatureVector  = FeatureVector_(FeatureSort )
FeatureVectorM = FeatureVector_(z3.IntSort())

In [28]:
some_fv3 = FeatureVector3( "some_fv3" )
some_fvM = FeatureVectorM("some_fvM")

In [29]:
z3.solve(some_fv3[FA] == plus, some_fv3[FB] == minus)

[some_fv3 = Store(K(Feature3, +), FB, -)]


In [30]:
modelFor_some_fv3 = solve_([some_fv3[FA] == plus, some_fv3[FB] == minus])

concretize(modelFor_some_fv3, some_fv3, FeatureSort3_constants)

{'FA': '+', 'FB': '-', 'FC': '+'}

## Parallel symbolic vectors

In [31]:
# NUM_FEATURES = M = 3
# D = 